In [71]:
# Importing the required libraries
import requests
import pandas as pd
from bs4 import BeautifulSoup

# Downloading contents of the web page
url = "https://drugcheckingbc.ca/results/"
data = requests.get(url).text

# Creating BeautifulSoup object
soup = BeautifulSoup(data, 'html.parser')

# Find tables
table = soup.find('table')

# Defining of the dataframe
df = pd.DataFrame(columns=['DATE', 'SITE', 'CITY/TOWN', 'EXPECTED SUBSTANCE', 'FENTANYL TEST STRIP', 'BENZO TEST STRIP', 'FTIR SPECTROMETER'])

# Collecting data
for row in table.tbody.find_all('tr'):    
    # Find all data for each column
    columns = row.find_all('td')
    
    if(columns != []):
        DATE = columns[0].text.strip()
        SITE = columns[1].text.strip()
        CITY_TOWN = columns[2].text.strip()
        EXPECTED_SUBSTANCE = columns[3].text.strip()
        FENTANYL_TEST_STRIP = columns[4].text.strip()
        BENZO_TEST_STRIP = columns[5].text.strip()
        FTIR_SPECTROMETER = str(columns[6])[59:-5].split('<br/>')

        row = [{'DATE': DATE,  'SITE': SITE, 'CITY/TOWN': CITY_TOWN, 'EXPECTED SUBSTANCE': EXPECTED_SUBSTANCE, 'FENTANYL TEST STRIP': FENTANYL_TEST_STRIP, 'BENZO TEST STRIP': BENZO_TEST_STRIP, 'FTIR SPECTROMETER': FTIR_SPECTROMETER}]
        df_new = pd.DataFrame.from_records(row)
        df = pd.concat([df, df_new])
        
df = df.reset_index(drop=True)

# make one column per drug
results = pd.DataFrame(df['FTIR SPECTROMETER'].values.tolist()).add_prefix('FTIR SPECTROMETER ')
df = pd.concat([df,results], axis=1).drop(columns='FTIR SPECTROMETER')  

df.head()

,DATE,SITE,CITY/TOWN,EXPECTED SUBSTANCE,FENTANYL TEST STRIP,BENZO TEST STRIP,FTIR SPECTROMETER 0,FTIR SPECTROMETER 1,FTIR SPECTROMETER 2,FTIR SPECTROMETER 3
0,2022-11-30,Outreach Urban Health,Kelowna,Down (Unknown Opioid),Positive,Positive,Erythritol,Caffeine,Fentanyl,None
1,2022-11-30,Burdock House (Penticton),Penticton,Crack Cocaine,Negative,N/A,Cocaine Base,None,None,None
2,2022-11-30,Get Your Drugs Tested,Vancouver,Unknown,Negative,Negative,MDA,Uncertain Match,None,None
3,2022-11-30,Get Your Drugs Tested,Vancouver,GHB,Negative,N/A,GHB (Wet),Uncertain Match,None,None
4,2022-11-30,Insite,Vancouver,Methamphetamine,Negative,N/A,Methamphetamine,Dimethyl Sulfone,None,None


In [56]:
# loading more rows

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

driver = webdriver.Chrome()
driver.get("https://drugcheckingbc.ca/results/")

button = driver.find_element(By.CSS_SELECTOR, ".sbcd-filters-title")
driver.execute_script("arguments[0].click();", button)